In [61]:
import pandas as pd
from datetime import datetime

Requires the Napa_weather_report.csv


1. drop the column because the CSV added a column - it made "Station" the index
2. drop type of percipitation - is completely null
3. drop air_temp_obs - is completely null
4. drop type of wx - is completely null
5. drop bulb temperate - is completely null
6. pan_evap - is completely null

In [2]:
## Read the CSV
napa = pd.read_csv('Napa_weather_report.csv')

## Rename the columns
napa.rename(columns={'Station':'date','Date':'time','Time':'precipitation','Precip':'type','type':'air_temp_max',
                    'Air max':'air_temp_min','min':'air_temp_obs','obs':'wx','Wx':'wind_dir','Wind dir':'wind_speed',
                    'speed':'bulb_temp_wet','Bulb wet':'bulb_temp_dry','dry':'soil_temp_max','Soil max':'soil_temp_min',
                    'min.1':'pan_evap','Evap':'solar_radiation','Solar':'evapotranspiration','ETo':'hum_max','RH max':'hum_min',
                    'min.2':'drop'}, inplace=True)


napa['date'] = pd.to_datetime(napa['date'], format='%Y%m%d')  # Convert 'date' to datetime format
napa.insert(0, 'year', napa['date'].dt.year)   
napa.insert(1, 'month', napa['date'].dt.month) 
napa.insert(2, 'day', napa['date'].dt.day) 
    

napa.drop(columns=['drop', 'type', 'air_temp_obs', 'wx', 'bulb_temp_wet','bulb_temp_dry','pan_evap'],inplace=True)

napa

,year,month,day,date,time,precipitation,air_temp_max,air_temp_min,wind_dir,wind_speed,soil_temp_max,soil_temp_min,solar_radiation,evapotranspiration,hum_max,hum_min
OAKVILLE.A,1989,1,1,1989-01-01,800,0.00,54,32,NaN,NaN,NaN,NaN,126,NaN,NaN,NaN
OAKVILLE.A,1989,1,2,1989-01-02,800,0.00,60,33,NaN,NaN,NaN,NaN,152,NaN,NaN,NaN
OAKVILLE.A,1989,1,3,1989-01-03,800,0.00,61,29,NaN,NaN,NaN,NaN,181,NaN,NaN,NaN
OAKVILLE.A,1989,1,4,1989-01-04,800,0.58,51,29,NaN,NaN,NaN,NaN,161,NaN,NaN,NaN
OAKVILLE.A,1989,1,5,1989-01-05,800,0.00,48,29,NaN,NaN,NaN,NaN,177,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OAKVILLE.A,2024,1,18,2024-01-18,2359,0.00,61,41,NaN,NaN,52.0,51.0,151,0.03,97.0,61.0
OAKVILLE.A,2024,1,19,2024-01-19,2359,0.62,57,47,NaN,NaN,52.0,51.0,68,0.01,96.0,72.0
OAKVILLE.A,2024,1,20,2024-01-20,2359,0.49,59,37,NaN,NaN,53.0,52.0,101,0.02,97.0,75.0
OAKVILLE.A,2024,1,21,2024-01-21,2359,0.29,58,39,NaN,NaN,NaN,NaN,147,NaN,NaN,NaN


The following query seeks to extract specific weather characteristics on an annual basis. By retrieving data on key stages such as budding, flowering, veraison, and harvest, users can gain deeper insights into the nuances of each growing season and, consequently, the grape composition for a given year.


In [69]:
tot_precipitation=[]
for year in range(1989, 2023):
    for month in range(1, 13):
        precipitation = napa[(napa['year'] == year) & (napa['month'] == month)]['precipitation'].sum()
        tot_precipitation.append({'year': year, 'month': month, 'precipitation': precipitation})
rain = pd.DataFrame(tot_precipitation)

rain.to_csv('rain.csv', index=False)


In [16]:
def GDD_bud(year):
    GDD_accumulation = 0
    t_base = 51
    for i in range(len(year)):
        GDD_accumulation += ((year['air_temp_max'][i] + year['air_temp_min'][i]) / 2) - t_base
        # print(f"Day {i+1}, GDD: {GDD_accumulation}")
        if GDD_accumulation >= 75:
            return year['date'][i].date().strftime('%Y-%m-%d')
    return None

In [17]:
def GDD_flower(year):
    GDD_accumulation = 0
    t_base = 51
    for i in range(len(year)):
        GDD_accumulation += ((year['air_temp_max'][i] + year['air_temp_min'][i]) / 2) - t_base
        # print(f"Day {i+1}, GDD: {GDD_accumulation}")
        if GDD_accumulation >= 750:
            return year['date'][i].date().strftime('%Y-%m-%d')
    return None

In [18]:
def GDD_veraison(year):
    GDD_accumulation = 0
    t_base = 51
    for i in range(len(year)):
        GDD_accumulation += ((year['air_temp_max'][i] + year['air_temp_min'][i]) / 2) - t_base
        if GDD_accumulation >= 1750:
            return year['date'][i].date().strftime('%Y-%m-%d')
    return None

In [71]:
def GDD_harvest(year):
    GDD_accumulation = 0
    t_base = 51
    for i in range(len(year)):
        GDD_accumulation += ((year['air_temp_max'][i] + year['air_temp_min'][i]) / 2) - t_base
        if GDD_accumulation >= 2500:
            return year['date'][i].date().strftime('%Y-%m-%d')            
    return str(year['date'][0].date().year)+'-12-31'

Putting it all together:

In [72]:
growing_season=pd.DataFrame()
years=[]
grow_begin=[]
flower=[]
veraison=[]
harvest=[]
for i in range(1989,2023):
    year = napa[napa['year']==i]
    grow_begin.append(GDD_bud(year))
    flower.append(GDD_flower(year))
    veraison.append(GDD_veraison(year))
    harvest.append(GDD_harvest(year)) 
    years.append(i)
growing_season.insert(0,'budding', grow_begin)
growing_season.insert(1,'flower', flower)
growing_season.insert(2,'veraison', veraison)
growing_season.insert(3,'harvest', harvest)
growing_season.insert(0, 'year', years)
growing_season

,year,budding,flower,veraison,harvest
0,1989,1989-04-28,1989-06-22,1989-08-16,1989-10-09
1,1990,1990-04-14,1990-06-12,1990-08-08,1990-09-24
2,1991,1991-04-29,1991-07-02,1991-09-01,1991-10-18
3,1992,1992-04-09,1992-06-02,1992-08-04,1992-09-19
4,1993,1993-04-27,1993-06-19,1993-08-13,1993-09-29
5,1994,1994-04-20,1994-06-23,1994-08-22,1994-10-13
6,1995,1995-03-14,1995-06-23,1995-08-12,1995-09-25
7,1996,1996-03-18,1996-06-03,1996-07-28,1996-09-06
8,1997,1997-03-25,1997-05-27,1997-07-23,1997-09-03
9,1998,1998-05-02,1998-07-08,1998-08-29,1998-10-24


Finally turning these into CSVs to use for deeper analysis.

In [66]:
napa.to_csv('napa_climate.csv', index=False)


In [73]:
growing_season.to_csv('growing_season.csv', index=False)